In [1]:
import datetime
import os
import mthree
import numpy as np
from qiskit import IBMQ
from qiskit.providers.ibmq import IBMQBackend
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit_research.mzm_generation import KitaevHamiltonianExperiment
from qiskit_research.mzm_generation.utils import pick_qubit_layout, orbital_combinations

In [2]:
# get backend

# hardware backend
# IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
# backend = provider.get_backend('ibmq_guadalupe')

# Aer simulator backend
# noise_model = NoiseModel.from_backend(backend)
# backend = AerSimulator(method='density_matrix', noise_model=noise_model)
backend = AerSimulator(method='statevector')

In [3]:
# Hamiltonian parameters
n_modes = 5
tunneling_values = [-1.0]
superconducting_values = [1.0]
chemical_potential_values = np.linspace(0.0, 3.0, num=5)
occupied_orbitals_list = list(orbital_combinations(n_modes, threshold=2))

# qubits
# qubits, backend_name, error_score = pick_qubit_layout(n_modes, [backend])
qubits = list(range(n_modes))
print(f'Using qubits {qubits}.')

# number of shots
shots = 100000

# number of shots for readout error calibration
readout_calibration_shots = 100000

# date
date = datetime.datetime.now().isoformat()

# create experiments
experiment = KitaevHamiltonianExperiment(
    experiment_id=f'{date}_{backend.name()}',
    backend=backend,
    readout_calibration_date=date,
    qubits=qubits,
    tunneling_values=tunneling_values,
    superconducting_values=superconducting_values,
    chemical_potential_values=chemical_potential_values,
    occupied_orbitals_list=occupied_orbitals_list,
    # dynamical_decoupling_sequences=['XY4pm']
)

Using qubits [0, 1, 2, 3, 4].


In [4]:
%%time

# schedule readout calibration
print('Scheduling readout calibration ...')
mit = mthree.M3Mitigation(backend)
mit.cals_from_system(qubits, shots=readout_calibration_shots)
# schedule experiment jobs
print(f'Scheduling experiment ...')
# limit number of circuits per job to avoid timeouts
backend.configuration().max_experiments = 150
data = experiment.run(shots=shots)
if isinstance(backend, IBMQBackend):
    data.save()

# save readout calibration
filename = f'data/readout_calibration/{backend.name()}/{date}.json'
os.makedirs(os.path.dirname(filename), exist_ok=True)
print(f'Waiting for readout calibration data ...')
mit.cals_to_file(filename)
print(f'Readout calibration saved.')

Scheduling readout calibration ...
Scheduling experiment ...
Waiting for readout calibration data ...
Readout calibration saved.
CPU times: user 15.9 s, sys: 124 ms, total: 16.1 s
Wall time: 16.4 s


In [5]:
# wait for experiment results
# data.block_for_results()

# TODO save data locally once it's supported
# See https://github.com/Qiskit/qiskit-experiments/issues/602

In [6]:
# TODO split off into analysis notebook once Qiskit Experiments supports saving data
# See https://github.com/Qiskit/qiskit-experiments/issues/602
import os
import matplotlib.pyplot as plt
import numpy as np
from qiskit import IBMQ
from qiskit_experiments.framework import ExperimentData
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_research.mzm_generation import KitaevHamiltonianAnalysis
np.set_printoptions(precision=3, suppress=True, linewidth=150)

In [10]:
%%time
experiment_id = data.metadata["experiment_id"]
qubits = data.metadata["qubits"]
n_modes = len(qubits)
tunneling_values = data.metadata["tunneling_values"]
superconducting_values = data.metadata["superconducting_values"]
chemical_potential_values = data.metadata["chemical_potential_values"]

# run analysis
analysis = KitaevHamiltonianAnalysis()
data = analysis.run(data)

energy_error_raw, energy_error_stddev_raw = data.analysis_results('energy_error_raw').value
energy_error_mem, energy_error_stddev_mem = data.analysis_results('energy_error_mem').value
energy_error_ps, energy_error_stddev_ps = data.analysis_results('energy_error_ps').value
energy_error_pur, energy_error_stddev_pur = data.analysis_results('energy_error_pur').value

bdg_energy_error_raw, bdg_energy_error_stddev_raw = data.analysis_results('bdg_energy_error_raw').value
bdg_energy_error_mem, bdg_energy_error_stddev_mem = data.analysis_results('bdg_energy_error_mem').value
bdg_energy_error_ps, bdg_energy_error_stddev_ps = data.analysis_results('bdg_energy_error_ps').value
bdg_energy_error_pur, bdg_energy_error_stddev_pur = data.analysis_results('bdg_energy_error_pur').value

CPU times: user 32.8 s, sys: 653 ms, total: 33.5 s
Wall time: 8.77 s
